In [190]:
import pandas as pd
import tabula as tb
import numpy as np


all = tb.read_pdf("./LUT.pdf", pages='all')



In [200]:
#initiation du dataframe final avec tt les valeurs

##fait la premiere page
#retire unités
df = all[0].drop(0)
#retire colonnes inutiles
del df['Number']
del df['Data']
del df['Reference']


#colonnes d'interet
columns = ['D', 'L', 'P', 'G','Xchf', 'DHin', 'CHF', 'Tin']
for col in columns:
    df[col] = df[col].astype(float)



for page in range(1,len(all)):
    #si manque 1 header on prend pas
    if len(all[page].keys()) < 11:
        pass
    else:
        #commence par enlever les infos du faux header
        header = all[page].keys()
        
            
        first_row = [[]]
        for i in header[2:10]:
            if i[-2:] == ' 0':
                i = i[:-2]
            first_row[0].append(float(i))
        rest = all[page].iloc[:,2:10].values
        for i in range(len(rest)):
            for j in range(len(rest[i])):
                if type(rest[i][j]) == str:
                    if rest[i][j][-2:] == ' 0':
                        rest[i][j] = rest[i][j][:-2]
                    rest[i][j] = float(rest[i][j])
        np_data_page = np.concatenate((first_row, rest))
        
        small_df = pd.DataFrame(np_data_page, columns =columns)


        df = pd.concat([df,small_df], ignore_index=True)

        


#print(df.iloc[0].values)
        #prend le header et et cree un data frame avec
            #ajoute tt les autres elem ligne par ligne
        #concat au tot


In [209]:
print(df)

           D      L        P       G    Xchf     DHin     CHF     Tin
0      0.004  0.396    100.0    77.5    0.84    317.0   442.0   23.94
1      0.004  0.396    100.0   142.7    0.79    317.0   757.0   23.94
2      0.004  0.396    100.0   203.9     0.7    317.0   978.0   23.94
3      0.004  0.396    100.0   271.8    0.73    317.0  1325.0   23.94
4      0.004  0.396    100.0   421.3    0.62    317.0  1798.0   23.94
...      ...    ...      ...     ...     ...      ...     ...     ...
24571  0.008    1.0  14808.0  1775.7  0.1832  123.858  1111.4  321.93
24572  0.008    1.0  14727.0   600.9  0.3532  323.342   819.5  289.89
24573  0.008    1.0  14727.0   608.0  0.4002  378.265   954.1  279.39
24574  0.008    1.0  14727.0   571.5  0.3839  570.264  1097.4  239.94
24575  0.008    1.0  14727.0   579.4  0.4044  587.056  1156.1  236.31

[24576 rows x 8 columns]


In [210]:
#save
df.to_csv('./csv_files/original_data.csv', index=False)

In [211]:
sort = pd.read_csv('./csv_files/original_data.csv')
print(sort)

           D      L        P       G    Xchf     DHin     CHF     Tin
0      0.004  0.396    100.0    77.5  0.8400  317.000   442.0   23.94
1      0.004  0.396    100.0   142.7  0.7900  317.000   757.0   23.94
2      0.004  0.396    100.0   203.9  0.7000  317.000   978.0   23.94
3      0.004  0.396    100.0   271.8  0.7300  317.000  1325.0   23.94
4      0.004  0.396    100.0   421.3  0.6200  317.000  1798.0   23.94
...      ...    ...      ...     ...     ...      ...     ...     ...
24571  0.008  1.000  14808.0  1775.7  0.1832  123.858  1111.4  321.93
24572  0.008  1.000  14727.0   600.9  0.3532  323.342   819.5  289.89
24573  0.008  1.000  14727.0   608.0  0.4002  378.265   954.1  279.39
24574  0.008  1.000  14727.0   571.5  0.3839  570.264  1097.4  239.94
24575  0.008  1.000  14727.0   579.4  0.4044  587.056  1156.1  236.31

[24576 rows x 8 columns]


In [212]:
#ancienne table 3 points de plus mais tt corrupt

sort['P'] = sort['P']*1000
sort['DHin'] = sort['DHin']*1000
sort['CHF'] = sort['CHF']*1000
sort['Tin'] = sort['Tin'] + 273.15


###FILTRE###
sort = sort.loc[sort['Xchf'] < 1 ]
sort = sort.loc[(sort['P'] <= 21000000) &  (sort['P'] >= 100000) ]
sort = sort.loc[(sort['D'] < 0.025) &  (sort['D'] > 0.003) ]
sort = sort.loc[(sort['G'] < 8000) &  (sort['G'] >= 0) ]

sort['L/D'] = sort['L']/sort['D'] 
sort = sort.loc[ ((sort['Xchf'] > 0) & ( sort['L/D']> 50))  |  ((sort['Xchf'] < 0) & ( sort['L/D'] > 25))]



print(sort)




           D      L           P       G    Xchf      DHin        CHF     Tin  \
0      0.004  0.396    100000.0    77.5  0.8400  317000.0   442000.0  297.09   
1      0.004  0.396    100000.0   142.7  0.7900  317000.0   757000.0  297.09   
2      0.004  0.396    100000.0   203.9  0.7000  317000.0   978000.0  297.09   
3      0.004  0.396    100000.0   271.8  0.7300  317000.0  1325000.0  297.09   
4      0.004  0.396    100000.0   421.3  0.6200  317000.0  1798000.0  297.09   
...      ...    ...         ...     ...     ...       ...        ...     ...   
24571  0.008  1.000  14808000.0  1775.7  0.1832  123858.0  1111400.0  595.08   
24572  0.008  1.000  14727000.0   600.9  0.3532  323342.0   819500.0  563.04   
24573  0.008  1.000  14727000.0   608.0  0.4002  378265.0   954100.0  552.54   
24574  0.008  1.000  14727000.0   571.5  0.3839  570264.0  1097400.0  513.09   
24575  0.008  1.000  14727000.0   579.4  0.4044  587056.0  1156100.0  509.46   

         L/D  
0       99.0  
1       9

In [213]:
sort.to_csv('./csv_files/sort_data.csv')

In [199]:

for i, j in enumerate(df['CHF'].values):
    if j <=0:
        print('aille')